# Imports + chemins + paramètres

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix

# ...
DATASET_DIR = "/kaggle/input/dataset-root/output"
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR   = os.path.join(DATASET_DIR, "val")
TEST_DIR  = os.path.join(DATASET_DIR, "test")

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
SEED = 42
NUM_CLASSES = 3  # alert, non_vigilant, tired

2025-11-17 20:11:50.475450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763410310.690810      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763410310.754976      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# # Data generators

In [2]:
def get_data_generators():
    # TRAIN: normalisation + augmentation légère
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
    )

    # VAL & TEST: juste normalisation (aucune augmentation)
    val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_gen = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",   # 3 classes → one-hot
        shuffle=True,
        seed=SEED
    )

    val_gen = val_test_datagen.flow_from_directory(
        VAL_DIR,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )

    test_gen = val_test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )

    print("Classes détectées :", train_gen.class_indices)
    print("Train:", train_gen.n, "| Val:", val_gen.n, "| Test:", test_gen.n)

    return train_gen, val_gen, test_gen

train_gen, val_gen, test_gen = get_data_generators()

Found 3124 images belonging to 3 classes.
Found 389 images belonging to 3 classes.
Found 394 images belonging to 3 classes.
Classes détectées : {'alert': 0, 'non_vigilant': 1, 'tired': 2}
Train: 3124 | Val: 389 | Test: 394


# Modèle 1 : CNN baseline

In [3]:
def build_model():
    model = keras.Sequential([
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(256, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),

        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

model = build_model()
model.summary()

I0000 00:00:1763410334.107604      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763410334.108257      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,811,459 (25.98 MB)

 Trainable params: 6,811,459 (25.98 MB)

 Non-trainable params: 0 (0.00 B)

* Callbacks + entraînement

In [4]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "models/fatigue_cnn_baseline.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

earlystop_cb = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    callbacks=[checkpoint_cb, earlystop_cb]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1763410339.802860     113 service.cc:148] XLA service 0x7f68e800d530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763410339.803949     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763410339.803969     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763410340.145577     113 cuda_dnn.cc:529] Loaded cuDNN version 90300


 2/98 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.2578 - loss: 1.1107 

I0000 00:00:1763410345.905779     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 826ms/step - accuracy: 0.3742 - loss: 1.0970
Epoch 1: val_accuracy improved from -inf to 0.41388, saving model to models/fatigue_cnn_baseline.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 98s 923ms/step - accuracy: 0.3744 - loss: 1.0969 - val_accuracy: 0.4139 - val_loss: 1.0534
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.4495 - loss: 1.0498
Epoch 2: val_accuracy improved from 0.41388 to 0.53728, saving model to models/fatigue_cnn_baseline.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 54s 556ms/step - accuracy: 0.4498 - loss: 1.0495 - val_accuracy: 0.5373 - val_loss: 0.9168
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.5172 - loss: 0.9513
Epoch 3: val_accuracy improved from 0.53728 to 0.61440, saving model to models/fatigue_cnn_baseline.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 54s 550ms/step - accuracy: 0.5174 - loss: 0.9510 - val_accuracy: 0.6144 - val_loss: 0.7940
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.6130 - loss: 0.838

# Modèle 2: EfficientNetB0 (ancienne version)

In [5]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effb0_preprocess

train_datagen_effb0 = ImageDataGenerator(
    preprocessing_function=effb0_preprocess
)

val_datagen_effb0 = ImageDataGenerator(
    preprocessing_function=effb0_preprocess
)

train_gen_effb0 = train_datagen_effb0.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen_effb0 = val_datagen_effb0.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

def build_effb0_model():
    base = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    base.trainable = False

    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

Found 3124 images belonging to 3 classes.
Found 389 images belonging to 3 classes.


* Callbacks + entraînement

In [6]:
effb0_model = build_effb0_model()

checkpoint_effnetb0 = keras.callbacks.ModelCheckpoint(
    "models/efficientnetb0.keras", save_best_only=True, monitor="val_accuracy"
)
history_effb0 = effb0_model.fit(
    train_gen_effb0,
    validation_data=val_gen_effb0,
    epochs=15,
    callbacks=[checkpoint_effnetb0,earlystop_cb]
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 66s 473ms/step - accuracy: 0.3386 - loss: 1.2198 - val_accuracy: 0.4242 - val_loss: 1.1111
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 282ms/step - accuracy: 0.3908 - loss: 1.1324 - val_accuracy: 0.4936 - val_loss: 1.0361
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 273ms/step - accuracy: 0.4846 - loss: 1.0308 - val_accuracy: 0.5424 - val_loss: 0.9799
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 273ms/step - accuracy: 0.5236 - loss: 0.9791 - val_accuracy: 0.5578 - val_loss: 0.9364
Epoch 5/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 276ms/step - accuracy: 0.5739 - loss: 0.9283 - val_accuracy: 0.6272 - val_loss: 0.8977
Epoch 6/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 271ms/step - accuracy: 0.6007 - loss: 0.8894 - val_accuracy: 0.6735 - val_loss: 0.8652
Epoch 7/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 275ms/step - accuracy: 0.6130 - loss: 0.8731 - val_accuracy: 0.6812 - val_loss: 0.8376
Epoch 8/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 270m

# Modèle 3 : EfficientNetV2B0 (Transfer Learning)

In [7]:
from tensorflow.keras.applications import EfficientNetV2B0

def build_efficientnet_model():
    # Base pré-entraînée sur ImageNet
    base_model = EfficientNetV2B0(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    # On gèle la base au début (on ne l'entraîne pas)
    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

effnet_model = build_efficientnet_model()
effnet_model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,923,155 (22.60 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

* Callbacks + entraînement

In [8]:
# Checkpoint spécifique pour EfficientNet
checkpoint_effnet = keras.callbacks.ModelCheckpoint(
    "models/fatigue_effnet_best.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

history_effnet = effnet_model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    callbacks=[checkpoint_effnet, earlystop_cb]  # on réutilise earlystop_cb
)

Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 602ms/step - accuracy: 0.3472 - loss: 1.1136
Epoch 1: val_accuracy improved from -inf to 0.36761, saving model to models/fatigue_effnet_best.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 95s 741ms/step - accuracy: 0.3472 - loss: 1.1136 - val_accuracy: 0.3676 - val_loss: 1.0971
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy: 0.3745 - loss: 1.1035
Epoch 2: val_accuracy did not improve from 0.36761
98/98 ━━━━━━━━━━━━━━━━━━━━ 53s 540ms/step - accuracy: 0.3744 - loss: 1.1035 - val_accuracy: 0.3676 - val_loss: 1.0958
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - accuracy: 0.3344 - loss: 1.1108
Epoch 3: val_accuracy did not improve from 0.36761
98/98 ━━━━━━━━━━━━━━━━━━━━ 53s 543ms/step - accuracy: 0.3345 - loss: 1.1107 - val_accuracy: 0.3676 - val_loss: 1.0955
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.3416 - loss: 1.1135
Epoch 4: val_accuracy did not improve from 0.36761
98/98 ━━━━━━━━━━━━━━━━━━━━ 53s 544ms/step - acc

- Évaluation

In [9]:
best_effnet = keras.models.load_model("models/fatigue_effnet_best.keras")

test_loss, test_acc = best_effnet.evaluate(test_gen)
print(f"[EfficientNetV2B0] Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}")

y_prob = best_effnet.predict(test_gen)
y_pred = np.argmax(y_prob, axis=1)
y_true = test_gen.classes

print("\nClass indices:", test_gen.class_indices)

print("\nClassification report (EfficientNet):")
print(classification_report(
    y_true,
    y_pred,
    target_names=list(test_gen.class_indices.keys())
))

print("Confusion matrix (EfficientNet):")
print(confusion_matrix(y_true, y_pred))

13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 798ms/step - accuracy: 0.1261 - loss: 1.1142
[EfficientNetV2B0] Test Loss: 1.0971 - Test Accuracy: 0.3680
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 648ms/step

Class indices: {'alert': 0, 'non_vigilant': 1, 'tired': 2}

Classification report (EfficientNet):
              precision    recall  f1-score   support

       alert       0.00      0.00      0.00       126
non_vigilant       0.00      0.00      0.00       123
       tired       0.37      1.00      0.54       145

    accuracy                           0.37       394
   macro avg       0.12      0.33      0.18       394
weighted avg       0.14      0.37      0.20       394

Confusion matrix (EfficientNet):
[[  0   0 126]
 [  0   0 123]
 [  0   0 145]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Modèle 4 : MobileNetV2 (modèle léger)

In [10]:
from tensorflow.keras.applications import MobileNetV2

def build_mobilenet_model():
    base_model = MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    base_model.trainable = False  # on gèle la base

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

mobilenet_model = build_mobilenet_model()
mobilenet_model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

* Callbacks + entraînement

In [11]:
checkpoint_mobilenet = keras.callbacks.ModelCheckpoint(
    "models/fatigue_mobilenet_best.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

history_mobilenet = mobilenet_model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    callbacks=[checkpoint_mobilenet, earlystop_cb]
)

Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.3404 - loss: 1.3873
Epoch 1: val_accuracy improved from -inf to 0.36761, saving model to models/fatigue_mobilenet_best.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 69s 626ms/step - accuracy: 0.3405 - loss: 1.3866 - val_accuracy: 0.3676 - val_loss: 1.1496
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.3729 - loss: 1.2182
Epoch 2: val_accuracy improved from 0.36761 to 0.42416, saving model to models/fatigue_mobilenet_best.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 53s 545ms/step - accuracy: 0.3730 - loss: 1.2180 - val_accuracy: 0.4242 - val_loss: 1.0660
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.4018 - loss: 1.1699
Epoch 3: val_accuracy improved from 0.42416 to 0.46015, saving model to models/fatigue_mobilenet_best.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 54s 548ms/step - accuracy: 0.4019 - loss: 1.1697 - val_accuracy: 0.4602 - val_loss: 1.0098
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - accuracy: 0.4

- Évaluation

In [12]:
best_mobilenet = keras.models.load_model("models/fatigue_mobilenet_best.keras")

test_loss, test_acc = best_mobilenet.evaluate(test_gen)
print(f"[MobileNetV2] Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}")

y_prob = best_mobilenet.predict(test_gen)
y_pred = np.argmax(y_prob, axis=1)
y_true = test_gen.classes

print("\nClassification report (MobileNet):")
print(classification_report(
    y_true,
    y_pred,
    target_names=list(test_gen.class_indices.keys())
))

print("Confusion matrix (MobileNet):")
print(confusion_matrix(y_true, y_pred))

13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 436ms/step - accuracy: 0.7148 - loss: 0.7495
[MobileNetV2] Test Loss: 0.7520 - Test Accuracy: 0.7005
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 445ms/step

Classification report (MobileNet):
              precision    recall  f1-score   support

       alert       0.65      0.71      0.68       126
non_vigilant       0.64      0.71      0.67       123
       tired       0.83      0.69      0.75       145

    accuracy                           0.70       394
   macro avg       0.71      0.70      0.70       394
weighted avg       0.71      0.70      0.70       394

Confusion matrix (MobileNet):
[[ 89  26  11]
 [ 26  87  10]
 [ 22  23 100]]


# Modèle 5: ResNet50V2

In [13]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications import ResNet50V2

train_datagen_res = ImageDataGenerator(
    preprocessing_function=resnet_preprocess,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen_res = ImageDataGenerator(
    preprocessing_function=resnet_preprocess
)

train_gen_res = train_datagen_res.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen_res = val_datagen_res.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

def build_resnet_model():
    base = ResNet50V2(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    base.trainable = False

    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

resnet_model = build_resnet_model()

Found 3124 images belonging to 3 classes.
Found 389 images belonging to 3 classes.
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


* Callbacks + entraînement

In [14]:
# ResNet50V2
checkpoint_resnet = keras.callbacks.ModelCheckpoint(
    "models/resnet_best.keras", save_best_only=True, monitor="val_accuracy"
)

history_resnet = resnet_model.fit(
    train_gen_res,
    validation_data=val_gen_res,
    epochs=15,
    callbacks=[checkpoint_resnet, earlystop_cb]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 80s 672ms/step - accuracy: 0.3183 - loss: 1.4488 - val_accuracy: 0.4550 - val_loss: 1.1343
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 54s 552ms/step - accuracy: 0.4003 - loss: 1.2286 - val_accuracy: 0.5167 - val_loss: 1.0271
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 55s 561ms/step - accuracy: 0.4611 - loss: 1.1390 - val_accuracy: 0.5424 - val_loss: 0.9563
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 55s 561ms/step - accuracy: 0.4966 - loss: 1.0382 - val_accuracy: 0.5758 - val_loss: 0.9058
Epoch 5/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 55s 564ms/step - accuracy: 0.5277 - loss: 0.9786 - val_accuracy: 0.6093 - val_loss: 0.8671
Epoch 6/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 55s 565ms/step - accuracy: 0.5520 - loss: 0.9377 - val_accuracy: 0.6427 - val_loss: 0.8282
Epoch 7/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 56s 568ms/step - accuracy: 0.5806 - loss: 0.9067 - val_accuracy: 0.6555 - val_loss: 0.7980
Epoch 8/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 54s 545ms/step - accuracy: 0.5801 - loss: 0.8910 - val_accu

# Modèle 6: DenseNet121

In [15]:
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications import DenseNet121

train_datagen_dense = ImageDataGenerator(
    preprocessing_function=densenet_preprocess,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen_dense = ImageDataGenerator(
    preprocessing_function=densenet_preprocess
)

train_gen_dense = train_datagen_dense.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen_dense = val_datagen_dense.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

def build_densenet_model():
    base = DenseNet121(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    base.trainable = False

    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

densenet_model = build_densenet_model()

Found 3124 images belonging to 3 classes.
Found 389 images belonging to 3 classes.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


- Entraînement

In [16]:
checkpoint_dense = keras.callbacks.ModelCheckpoint(
    "models/densenet_best.keras", save_best_only=True, monitor="val_accuracy"
)

history_dense = densenet_model.fit(
    train_gen_dense,
    validation_data=val_gen_dense,
    epochs=15,
    callbacks=[checkpoint_dense, earlystop_cb]
)

Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 112s 877ms/step - accuracy: 0.3387 - loss: 1.4451 - val_accuracy: 0.3470 - val_loss: 1.2030
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 582ms/step - accuracy: 0.3609 - loss: 1.3049 - val_accuracy: 0.3830 - val_loss: 1.1505
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 583ms/step - accuracy: 0.3812 - loss: 1.2383 - val_accuracy: 0.4139 - val_loss: 1.1132
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 578ms/step - accuracy: 0.4081 - loss: 1.1771 - val_accuracy: 0.4344 - val_loss: 1.0756
Epoch 5/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 578ms/step - accuracy: 0.4195 - loss: 1.1602 - val_accuracy: 0.4396 - val_loss: 1.0429
Epoch 6/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 580ms/step - accuracy: 0.4380 - loss: 1.1126 - val_accuracy: 0.4679 - val_loss: 1.0137
Epoch 7/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 56s 574ms/step - accuracy: 0.4348 - loss: 1.1033 - val_accuracy: 0.5039 - val_loss: 0.9885
Epoch 8/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 57s 577ms/step - accuracy: 0.4628 - loss: 1.0512 - val_acc

# Modèle 6: InceptionV3

In [17]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess

train_datagen_inc = ImageDataGenerator(
    preprocessing_function=inception_preprocess
)

val_datagen_inc = ImageDataGenerator(
    preprocessing_function=inception_preprocess
)

train_gen_inc = train_datagen_inc.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen_inc = val_datagen_inc.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

def build_inception_model():
    base = InceptionV3(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )
    base.trainable = False

    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

inception_model = build_inception_model()

Found 3124 images belonging to 3 classes.
Found 389 images belonging to 3 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
# InceptionV3
checkpoint_inc = keras.callbacks.ModelCheckpoint(
    "models/inceptionv3_best.keras", save_best_only=True, monitor="val_accuracy"
)

history_inception = inception_model.fit(
    train_gen_inc,
    validation_data=val_gen_inc,
    epochs=15,
    callbacks=[checkpoint_inc, earlystop_cb]
)

Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 63s 487ms/step - accuracy: 0.3506 - loss: 1.3687 - val_accuracy: 0.4499 - val_loss: 1.0698
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 296ms/step - accuracy: 0.4184 - loss: 1.2167 - val_accuracy: 0.5270 - val_loss: 0.9807
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 289ms/step - accuracy: 0.4734 - loss: 1.1183 - val_accuracy: 0.5424 - val_loss: 0.9088
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 292ms/step - accuracy: 0.4810 - loss: 1.0522 - val_accuracy: 0.6170 - val_loss: 0.8503
Epoch 5/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 290ms/step - accuracy: 0.5323 - loss: 0.9685 - val_accuracy: 0.6658 - val_loss: 0.8054
Epoch 6/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 288ms/step - accuracy: 0.5637 - loss: 0.9118 - val_accuracy: 0.6838 - val_loss: 0.7696
Epoch 7/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 293ms/step - accuracy: 0.6002 - loss: 0.8653 - val_accuracy: 0.7198 - val_loss: 0.7378
Epoch 8/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 291ms/step - accuracy: 0.6005 - loss: 0.8692 - val_accu

# Rapport (Comparaison finale des modèles)

In [19]:
print("=== Résumé des performances sur le test set ===")

# 0. CNN baseline
cnn_best = keras.models.load_model("models/fatigue_cnn_baseline.keras")
_, acc_cnn = cnn_best.evaluate(test_gen, verbose=0)

# 1. EfficientNetB0 (ancienne version)
effb0_best = keras.models.load_model("models/efficientnetb0.keras")
_, acc_effb0 = effb0_best.evaluate(test_gen, verbose=0)

# 2. EfficientNetV2B0 (nouvelle version)
effv2_best = keras.models.load_model("models/fatigue_effnet_best.keras")
_, acc_effv2 = effv2_best.evaluate(test_gen, verbose=0)

# 3. MobileNetV2
mobilenet_best = keras.models.load_model("models/fatigue_mobilenet_best.keras")
_, acc_mobilenet = mobilenet_best.evaluate(test_gen, verbose=0)

# 4. ResNet50V2
resnet_best = keras.models.load_model("models/resnet_best.keras")
_, acc_resnet = resnet_best.evaluate(test_gen, verbose=0)

# 5. DenseNet121
densenet_best = keras.models.load_model("models/densenet_best.keras")
_, acc_densenet = densenet_best.evaluate(test_gen, verbose=0)

# 6. InceptionV3
inception_best = keras.models.load_model("models/inceptionv3_best.keras")
_, acc_inception = inception_best.evaluate(test_gen, verbose=0)

# Affichage formaté
results = {
    "CNN baseline"      : acc_cnn,
    "EfficientNetB0"    : acc_effb0,
    "EfficientNetV2B0"  : acc_effv2,
    "MobileNetV2"       : acc_mobilenet,
    "ResNet50V2"        : acc_resnet,
    "DenseNet121"       : acc_densenet,
    "InceptionV3"       : acc_inception,
}

for name, acc in results.items():
    print(f"{name:18s} : {acc:.4f}")

=== Résumé des performances sur le test set ===
CNN baseline       : 0.9772
EfficientNetB0     : 0.3680
EfficientNetV2B0   : 0.3680
MobileNetV2        : 0.7005
ResNet50V2         : 0.7411
DenseNet121        : 0.5711
InceptionV3        : 0.7893
